In [1]:
!nvcc --version
# UPDATE SPACY INSTALL TO MATCH CUDA VERSION
!pip install -U spacy
!pip install -U spacy[cuda111]
!python -m spacy download en_core_web_trf

import srsly
import spacy
from spacy.training import docs_to_json, offsets_to_biluo_tags, biluo_tags_to_spans
from random import shuffle

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
     |████████████████████████████████| 6.0 MB 12.6 MB/s 
     |████████████████████████████████| 628 kB 58.9 MB/s 
     |████████████████████████████████| 181 kB 62.4 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 451 kB 60.8 MB/s 
     |████████████████████████████████| 10.1 MB 55.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled th

# Convert filtered data

In [3]:
# CONVERT TO IOB
train = srsly.read_json(r'/content/drive/MyDrive/4th year project/train_home.json')

shuffle(train)

train = train[:int(0.1*len(train))]

nlp = spacy.load('en_core_web_trf')
docs = []
for text, annot in train:
    doc = nlp(text)
    tags = offsets_to_biluo_tags(doc, annot['entities'])
    entities = biluo_tags_to_spans(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json(r'/content/drive/MyDrive/4th year project/train_home_iob.json', [docs_to_json(docs)])

dev = srsly.read_json(r'/content/drive/MyDrive/4th year project/dev_home.json')

dev = dev[:int(0.1*len(dev))]

docs = []
for text, annot in dev:
    doc = nlp(text)
    tags = offsets_to_biluo_tags(doc, annot['entities'])
    entities = biluo_tags_to_spans(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json(r'/content/drive/MyDrive/4th year project/dev_home_iob.json', [docs_to_json(docs)])

In [4]:
# CONVERT TO SPACY BINARY 
!python -m spacy convert "/content/drive/MyDrive/4th year project/train_home_iob.json" "/content/drive/MyDrive/4th year project" -t spacy
!python -m spacy convert "/content/drive/MyDrive/4th year project/dev_home_iob.json" "/content/drive/MyDrive/4th year project" -t spacy

✔ Generated output file (6447 documents): /content/drive/MyDrive/4th
year project/train_home_iob.spacy
✔ Generated output file (716 documents): /content/drive/MyDrive/4th
year project/dev_home_iob.spacy


# Create spacy config

In [8]:
!python -m spacy init fill-config "/content/drive/MyDrive/4th year project/home_bert_ner_config.cfg"
!python -m spacy debug data "/content/drive/MyDrive/4th year project/home_bert_ner_config.cfg"

[paths]
train = "/content/drive/MyDrive/4th year project/train_home_iob.spacy"
dev = "/content/drive/MyDrive/4th year project/dev_home_iob.spacy"
vectors = null
init_tok2vec = null

[system]
gpu_allocator = "pytorch"
seed = 0

[nlp]
lang = "en"
pipeline = ["transformer","ner"]
batch_size = 128
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.transformer]
factory = "transformer"
m

# Training model

In [11]:
!python -m spacy train -g 0 "/content/drive/MyDrive/4th year project/home_bert_ner_config.cfg" --output "/content/drive/MyDrive/4th year project"

ℹ Saving to output directory: /content/drive/MyDrive/4th year
project
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-03-13 17:12:19,510] [INFO] Set up nlp object from config
[2022-03-13 17:12:19,520] [INFO] Pipeline: ['transformer', 'ner']
[2022-03-13 17:12:19,524] [INFO] Created vocabulary
[2022-03-13 17:12:19,525] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be e

# Evaluate on unseen dev set

In [12]:
!python -m spacy evaluate -g 0 "/content/drive/MyDrive/4th year project/model-best" "/content/drive/MyDrive/4th year project/dev_home_iob.spacy" -o "/content/drive/MyDrive/4th year project/home_metrics.json" -dp "/content/drive/MyDrive/4th year project"

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   70.26 
NER R   66.67 
NER F   68.42 
SPEED   7147  


=============================== NER (per type) ===============================

          P       R       F
TYN   70.26   66.67   68.42

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
✔ Generated 25 parses as HTML
/content/drive/MyDrive/4th year project
✔ Saved results to /content/drive/MyDrive/4th year
project/home_metrics.json
